In [26]:
"""
Created on Mon February 23rd, 2022
@author: Eleftheria Chatzitheodoridou

Description: 
This script is designed to read an excel file that contains tumor boundary information of 
LGG cases from the local directory and check if the segmentator has extracted the tumor area. 
If it failed to do so, it is instructed to extract the (min, max) values from the excel 
file, find those indices on the NIfTI images, extract the 2D slices, and finally, save them
as .png files.

Note: This script was created on a Linux computer and some commands may not work on Windows/MAC OS.
"""

import os.path 
import numpy as np
import glob
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import re
from PIL import Image
from numpy import ndarray

# Folder where the created images will be saved in
out_path = r'/local/data1/elech646/Tumor_grade_classification/Slices'

# Create subfolders
if not os.path.exists(out_path + "/sagittal_grade_classification"):
    os.mkdir(out_path + "/sagittal_grade_classification")
    
if not os.path.exists(out_path + "/frontal_grade_classification"):
    os.mkdir(out_path + "/frontal_grade_classification")

if not os.path.exists(out_path + "/trans_grade_classification"):
    os.mkdir(out_path + "/trans_grade_classification")

# Add HGG path
sag_path = out_path + "/sagittal_grade_classification" + "/LGG"
fro_path = out_path + "/frontal_grade_classification" + "/LGG"
tra_path = out_path + "/trans_grade_classification" + "/LGG"

if not os.path.exists(sag_path):
    os.mkdir(sag_path)
    
if not os.path.exists(fro_path):
    os.mkdir(fro_path)
    
if not os.path.exists(tra_path):
    os.mkdir(tra_path)

filepath = []
# Scanning files and directories
for roots, dirs, files in os.walk("/local/data1/elech646/Tumor_grade_classification/LGG"):
    for name in files:
        if name.endswith((".nii.gz",".nii")):
                    filepath.append(roots + os.path.sep + name)
                
# Creating the images
for i, file in enumerate(filepath):
    img = nib.load(file)
    img_data = img.get_fdata()  # Getting data matrix
    patient = file.split("LGG/")[1].split("/")[0] + "/"
    if not os.path.exists(sag_path + "/" + patient):
        os.mkdir(sag_path + "/" + patient)
    if not os.path.exists(fro_path + "/" + patient):
        os.mkdir(fro_path + "/" + patient)
    if not os.path.exists(tra_path + "/" + patient):
        os.mkdir(tra_path + "/" + patient)
    
    # Importing the excel file with the manual checks for segmentation
    # Column names from Excel file: 
    # CDE_ID, CDE_TYPE, Laterality, Segmented, xmin, xmax, ymin, ymax, zmin, zmax
    annotations_LGG = pd.read_excel(r'/local/data1/elech646/Tumor_grade_classification/LGG case annotations.xlsx')


    # Check if the segmentator gave an annotation
    annotations_LGG = annotations_LGG[annotations_LGG.Segmented == 'NO'].reset_index(drop = True).\
    drop(["CDE_TYPE", "Laterality", "Segmented"], axis = 1)
        
    for idx in annotations_LGG.itertuples():
        tr_0 = int(idx.xmin)    
        tr_1 = int(idx.xmax)    
        fr_0 = int(idx.ymin)    
        fr_1 = int(idx.ymax)    
        sag_0 = int(idx.zmin)   
        sag_1 = int(idx.zmax)   
    
    # Creating the images in the Sagittal Plane (yz)
    img_sag = np.rot90(img_data, axes = (1, 2)) # yz plane sagittal
    img_sag = np.flip(img_sag, 0) # flip the image left/right since mango was used
    for sag in range(sag_0, sag_1 + 1):
        perc = int(((sag - sag_0)/(sag_1 - sag_0))*100) # Percentage along the selected slices
        tmp = img_sag[sag,:,:]
        min_v = img_sag.min()
        max_v = img_sag.max()
        
        # Normalize image
        tmp_norm = (255*(tmp - min_v) / max_v).astype(np.uint8) 
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii", 1)[0].split("_")[-1]
        
        # Name the files
        title = sag_path + "/" + patient + patient[:-1] + '_' + modality + str(perc)
        
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
        
        # Create output path
        out_path_f = out_path + "/"
        
        # Save images
        im.save(os.path.join(out_path_f, title + ".png"))
        
    # Creating the images in the Frontal Plane (xz)
    img_fr = np.rot90(img_data, axes = (0,2)) # xz plane frontal
    for front in range(fr_0, fr_1 + 1):
        perc = int(((front - fr_0) /(fr_1 - fr_0))*100) # Percentage along the selected slices
        tmp = img_fr[:,front,:]
        min_v = img_fr.min()
        max_v = img_fr.max()
        
        # Normalize image
        tmp_norm = (255*(tmp - min_v) / max_v).astype(np.uint8) 
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii", 1)[0].split("_")[-1]
        
        # Name the files
        title = fro_path + "/" + patient + patient[:-1] + '_' + modality + str(perc)
        
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
        
        # Create output file path
        out_path_f = out_path + "/"
        
        # Save images
        im.save(os.path.join(out_path_f, title + ".png"))  
    
    # Creating the images in the Transversal/Axial Plane (xy)
    img_tr = np.rot90(img_data, 3, axes = (0,1)) # xy plane transversal
    x,y,z = img_tr.shape
    
    # Inverting slices upside/down since mango was used
    tr_0 = z - tr_0
    tr_1 = z - tr_1
    
    for transv in range(tr_0, tr_1):
        perc = int(((transv - tr_0) / (tr_1 - tr_0))*100) # Percentage along the selected slices
        tmp = img_tr[:,:,transv]
        min_v = img_tr.min()
        max_v = img_tr.max()
        
        # Normalize image
        tmp_norm = (255*(tmp - min_v) / max_v).astype(np.uint8) 
        
        # Add modality        
        modality = filepath[i].split("_",1)[1].split(".nii", 1)[0].split("_")[-1]
        
        # Name the files
        title = tra_path + "/" + patient + patient[:-1] + '_' + modality + str(perc)
        
        # Convert to RGB
        im = Image.fromarray(tmp_norm).convert('RGB')
        
        # Create output file path
        out_path_f = out_path + "/" 
        
        # Save images
        im.save(os.path.join(out_path_f, title + ".png"))